In [ ]:
%pip install opencv-python numpy
%pip install kagglehub

: 

In [ ]:
# import kagglehub # we are using an api to import kagglehub's dataset

# path = kagglehub.dataset_download("jafarhussain786/human-emotionshappy-faces")

# path

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image as PILImage
import io
from IPython.display import display, Image


path = "./processed_images"

def resize_image(image, target_size=(224, 224)):
    h, w = image.shape[:2]
    scale = min(target_size[0] / h, target_size[1] / w)
    new_w = int(w * scale)
    new_h = int(h * scale)

    resized_image = cv2.resize(image, (new_w, new_h))
    canvas = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)
    top = (target_size[0] - new_h) // 2
    left = (target_size[1] - new_w) // 2
    canvas[top:top + new_h, left:left + new_w] = resized_image

    return canvas


def set_black_background(image, threshold=20):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray_image, threshold, 255, cv2.THRESH_BINARY_INV)
    image[mask == 255] = [0, 0, 0]
    return image


def preprocess_image(image, grayscale=True, target_size=(224, 224)):
    image = resize_image(image, target_size)
    image = set_black_background(image)
    if grayscale:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image


def random_rotation(image):
    angle = np.random.uniform(-30, 30)
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, M, (w, h))

def random_flip(image):
    if np.random.rand() > 0.5:
        return cv2.flip(image, 1)  
    return image

def random_brightness(image):
    factor = np.random.uniform(0.5, 1.5)
    return cv2.convertScaleAbs(image, alpha=factor, beta=0)

def random_zoom(image):
    h, w = image.shape[:2]
    zoom = np.random.uniform(0.8, 1.2)
    new_h, new_w = int(h * zoom), int(w * zoom)
    image = cv2.resize(image, (new_w, new_h))
    if new_h < h or new_w < w:
        top = (h - new_h) // 2
        left = (w - new_w) // 2
        cropped = image[top:top + new_h, left:left + new_w]
    else:
        cropped = image[:h, :w]
    return cropped


def augment_image(image):
    image = random_rotation(image)
    image = random_flip(image)
    image = random_brightness(image)
    image = random_zoom(image)
    return image


def load_and_display_images(num_images=1):
    image_files = [f for f in os.listdir(path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    for i, image_file in enumerate(image_files[:num_images]):
        image_path = os.path.join(path, image_file)
        print(f"Loading image: {image_path}")
        
        image = cv2.imread(image_path)
        original_image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
       
        processed_image = preprocess_image(image)
        augmented_image = augment_image(processed_image)

        
        display(Image.fromarray(original_image_rgb))
        display(Image.fromarray(cv2.cvtColor(augmented_image, cv2.COLOR_BGR2RGB)))


load_and_display_images(num_images=5)

# def show_images(num_images=1):
#     for i, image_file in enumerate(image_files[:num_images]):
#         image_path = os.path.join(path, image_file)
#         print(f"Using image {i+1} for testing: {image_path}")

#         image = cv2.imread(image_path)

#         original_image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         processed_image = preprocess_image(image)

#         processed_image_rgb = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)

#         original_img_pil = PILImage.fromarray(original_image_rgb)
#         buf_original = io.BytesIO()
#         original_img_pil.save(buf_original, format='JPEG')
#         buf_original.seek(0)

#         processed_img_pil = PILImage.fromarray(processed_image_rgb)
#         buf_processed = io.BytesIO()
#         processed_img_pil.save(buf_processed, format='JPEG')
#         buf_processed.seek(0)

#         print(f"Original Image {i+1}:")
#         display(Image(data=buf_original.getvalue()))

#         print(f"Image {i+1} after ETL:")
#         display(Image(data=buf_processed.getvalue()))

# show_images(num_images_to_display)


def save_processed_images(num_images=1, output_dir="./processed_images"):
    os.makedirs(output_dir, exist_ok=True)
    image_files = [f for f in os.listdir(path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    for i, image_file in enumerate(image_files[:num_images]):
        image_path = os.path.join(path, image_file)
        print(f"Processing image: {image_path}")

        image = cv2.imread(image_path)
        processed_image = preprocess_image(image)
        augmented_image = augment_image(processed_image)

        processed_image_rgb = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2RGB)
        output_file_path = os.path.join(output_dir, f"processed_{i + 1}.jpg")
        PILImage.fromarray(processed_image_rgb).save(output_file_path)
        print(f"Processed image saved to: {output_file_path}")

